In [1]:
%pip install chromospyce -q
%pip install pyBigWig -q
import pyBigWig
import chromospyce
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [22]:
# Initial experiment: color scale of maternal and paternal chromatin

# Read in data from Tan et al. 
data = pd.read_parquet('Tan3D.parquet', engine='pyarrow')

# Concatenate to only x, y, z coordinates + to numpy table

full_coords = data[["x","y","z"]]

full_coords = full_coords.to_numpy()

# Set up color scale
def get_mat_pat(chrom_str):
    if 'mat' in chrom_str:
        return 0.5
    return 1

# maternal is yellow, paternal chromatin is purple

color_values = [get_mat_pat(chrom_str) for chrom_str in data['chr']]


vc = {
    "color": {
        "values": color_values,
        "min": 0,
        "max": 1,
        "colorScale": "spectral"
    },
    "scale": 0.002,
    "links": True,
    "mark": "sphere"
}

chromospyce.Widget(full_coords, vc)

Widget(structure=b'\xff\xff\xff\xff\x80\x03\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0e\x00\x06\x00\x05\x00\x08\…

In [21]:
# Experiment #2: color scale by chromosome numbers 
# My goal for this one was to be able to see the locations of the paternal and maternal chromatin of the same chromosome
# However, there are not enough colors for this to be easy to look at


def get_chromosome_number(chrom_str):
    # For 'X' and 'Y' chromosomes
    if 'X' in chrom_str:
        return 23
    return int(chrom_str.split('(')[0])

# Create a list of color values based on chromosome number
color_values = [get_chromosome_number(chrom_str) for chrom_str in data['chr']]

vc = {
    "color": {
        "values": color_values,
        "min": 1,
        "max": 23,
        "colorScale": "Spectral"
    },
    "scale": 0.002,
    "links": True,
    "mark": "sphere"
}

chromospyce.Widget(full_coords, vc)

Widget(structure=b'\xff\xff\xff\xff\x80\x03\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0e\x00\x06\x00\x05\x00\x08\…

In [20]:
# Experiment 3: I wanted to pull out one chromosome
# I chose Chromosome 15 for arbitrary reasons
chrom15_pat = data[data['chr'] == '15(pat)']

chrom15_pat_coords = chrom15_pat[["x","y","z"]]

chrom15_pat_coords = chrom15_pat_coords.to_numpy()


numbers_chrom15 = list(range(0, len(chrom15_pat_coords)+1))

vc = {
    "color": {
        "values": numbers_chrom15,
        "min": 0,
        "max": len(chrom15_pat_coords)+1,
        "colorScale": "Spectral"
    },
    "scale": 0.01,
    "links": False,
    "mark": "sphere"
}

chromospyce.Widget(chrom15_pat_coords, vc)


Widget(structure=b'\xff\xff\xff\xffx\x03\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0e\x00\x06\x00\x05\x00\x08\x00…

In [19]:
# Experiment 4: pulling out the maternal and paternal chromatin of one chromosome
# used chromsome 21
chrom21_full = data[(data['chr'] == '21(mat)') | (data['chr'] == '21(pat)')]

chrom21_full_coords = chrom21_full[["x", "y", "z"]].to_numpy()

chrom21_mat = data[(data['chr']== '21(mat)')]
chrom21_mat_coords = chrom21_mat[["x", "y", "z"]].to_numpy()
chrom21_mat_coords = chrom21_mat[["coord"]].to_numpy()

print(max(chrom21_mat_coords) - min(chrom21_mat_coords))
print(len(chrom21_mat_coords))


# Reset index and store the result in a variable
highlight_indices_21 = chrom21_mat.reset_index(drop=True)

highlight_color = 0
default_color = 1

# Create the color_values with the correct range
color_values = [highlight_color if i in highlight_indices_21.index else default_color for i in range(len(chrom21_full_coords))]

vc = {
    "color": {
        "values": color_values,
        "min": 0,
        "max": 1,
        "colorScale": "Spectral"  # Consider if another scale is more appropriate
    },
    "scale": 0.01,
    "links": False,
    "mark": "sphere"
}

chromospyce.Widget(chrom21_full_coords, vc)


[38600000]
387


Widget(structure=b'\xff\xff\xff\xffx\x03\x00\x00\x10\x00\x00\x00\x00\x00\n\x00\x0e\x00\x06\x00\x05\x00\x08\x00…